# G202158002 Shin Won Hyung

## Ex 7.1

$\hat{\theta}_1 = \theta_0 - 0.1 \times \hat{g}_0 (\theta_0)$

$\hat{g}_0(\theta_0) = \frac{L(\theta_0 + \triangle_0) - L(\theta_0 -\triangle_0)}{2} [1/\triangle_{0,1}, 1/\triangle_{0,2}]^T$

$\triangle_0 \in \{(1,1), (1,-1), (-1,1), (-1,-1)\}$, $\theta_0 = [1,1]$

$\Rightarrow \hat{g}_0(\theta_0) \in \{6, 2, -2, -6\}$

$\therefore ~ \hat{\theta}_1 \in \{[0.4, 0.4]^T, [0.8, 0.8]^T, [1.2, 1.2]^T, [1.6, 1.6]^T\}$

## Ex 7.3

In [1]:
#%%
import numpy as np
from scipy.stats import bernoulli

#%%
## 7.3

def generate_lr(iter):
    return 0.5 / np.power(iter + 1 + 50, 0.602)

def generate_ck(iter):
    return 0.1 / np.power(iter + 1, 0.101)


def calculate_loss(theta, B, ck, nabla):
    measure = theta @ B @ B.T @ theta.T + 0.1 * np.sum((B.T @ theta.T) @ (B.T @ theta.T) * (B.T @ theta.T)) + 0.01 * np.sum((B.T @ theta.T) @ (B.T @ theta.T) * (B.T @ theta.T) @ (B.T @ theta.T))
    return measure + ck * nabla


def generate_nabla1():
    rv = bernoulli.rvs(size=1, p=0.5).item()
    if rv:
        return 1
    else:
        return -1


def generate_nabla1():
    rv = bernoulli.rvs(size=10, p=0.5)
    return rv + rv - 1


def generate_nabla2():
    return np.random.uniform(-np.sqrt(3), np.sqrt(3), 10)


def generate_nabla3():
    return np.random.normal(0, 1, 10)


def generate_perturb():
    return np.random.normal(0, 0.01, 1).item()


p = 10
iters = 2000
B = np.triu(np.ones([p,p])) / p

theta = np.ones(p)
for iter in range(iters):
    lr = generate_lr(iter)
    ck = generate_ck(iter)
    nabla = generate_nabla1()
    grad = ((calculate_loss(theta, B, ck, nabla)+generate_perturb()) - calculate_loss(theta, B, ck, -nabla)+generate_perturb()) / (2*ck) @ (1 / nabla)

    new_theta = theta - lr * grad

print(f"[Perturbation 1] theta: \n{new_theta}")


theta = np.ones(p)
for iter in range(iters):
    lr = generate_lr(iter)
    ck = generate_ck(iter)
    nabla = generate_nabla2()
    grad = ((calculate_loss(theta, B, ck, nabla)+generate_perturb()) - calculate_loss(theta, B, ck, -nabla)+generate_perturb()) / (2*ck) @ (1 / nabla)

    new_theta = theta - lr * grad

print(f"[Perturbation 2] theta: \n{new_theta}")


theta = np.ones(p)
for iter in range(iters):
    lr = generate_lr(iter)
    ck = generate_ck(iter)
    nabla = generate_nabla3()
    grad = ((calculate_loss(theta, B, ck, nabla)+generate_perturb()) - calculate_loss(theta, B, ck, -nabla)+generate_perturb()) / (2*ck) @ (1 / nabla)

    new_theta = theta - lr * grad

print(f"[Perturbation 3] theta: \n{new_theta}")


[Perturbation 1] theta: 
[0.94521053 0.94521053 0.94521053 0.94521053 0.94521053 0.94521053
 0.94521053 0.94521053 0.94521053 0.94521053]
[Perturbation 2] theta: 
[0.94857251 0.94857251 0.94857251 0.94857251 0.94857251 0.94857251
 0.94857251 0.94857251 0.94857251 0.94857251]
[Perturbation 3] theta: 
[0.94157611 0.94157611 0.94157611 0.94157611 0.94157611 0.94157611
 0.94157611 0.94157611 0.94157611 0.94157611]


## Ex 8.4

In [2]:
#%%
import numpy as np
from copy import copy
from scipy.stats import bernoulli
## 8.4
# %%
def inverse(permutation):
    permutation_copy = copy(permutation)
    change_list = permutation_copy[1:7]
    changed_list = change_list[-1::-1]

    permutation_copy[1:7] = changed_list
    return permutation_copy

def translate(permutation):
    permutation_copy = copy(permutation)
    change_list = permutation_copy[1:8]
    changed_list = [change_list[-1]] + change_list[:-1]

    permutation_copy[1:8] = changed_list
    return permutation_copy


def switch(permutation):
    permutation_copy = copy(permutation)
    change_list = permutation_copy[1:7]
    changed_list = [
        change_list[-1],
        change_list[1],
        change_list[-3],
        change_list[2],
        change_list[-2],
        change_list[0]
        ]
    permutation_copy[1:7] = changed_list

    return permutation_copy


def generate_costs():
    costs = np.random.uniform(0,1,np.math.factorial(9)) * 115 + 10
    cost_rows = []
    cum_i = 0
    for i in range(9,-1, -1):
        row = [0] * (10 - i) + costs[cum_i:cum_i + i].tolist()
        cost_rows.append(row)
        cum_i += i

    cost_matrix_ = np.stack(cost_rows)
    cost_matrix = cost_matrix_.T + cost_matrix_

    return cost_matrix


def calculate_cost(permutation, cost_matrix):
    permutation_reverse = copy(permutation)
    permutation_reverse.reverse()

    total_costs = 0
    for _ in range(9):
        i = permutation_reverse.pop()
        j = permutation_reverse[-1]
        total_costs += cost_matrix[i,j]

    return total_costs


permutation = [0,1,2,3,4,5,6,7,8,9]
cost_matrix = generate_costs()
events = [0]*75 + [1]*15 + [2]*15
T = 70
c_b = 1
lmbd = 0.95
repls = 5
iters = 8000


for repl in range(repls):
    np.random.seed(repl)
    permutation = [0,1,2,3,4,5,6,7,8,9]
    current_cost = calculate_cost(permutation, cost_matrix)

    for iter in range(iters):
        decay_factor = (iter+1) // 40
        T_iter = T * lmbd**(decay_factor)

        operation = np.random.choice(events, 1).item()
        if operation == 0:
            new_permutation = inverse(permutation)
            
        elif operation == 1:
            new_permutation = translate(permutation)

        elif operation == 2:
            new_permutation = switch(permutation)
        
        new_cost = calculate_cost(new_permutation, cost_matrix)

        delta = new_cost - current_cost

        if delta < 0:
            current_cost = new_cost
            permutation = new_permutation

        else:
            p = np.exp(-delta / (c_b * T_iter))
            trial = bernoulli.rvs(size=1, p=p).item()

            if trial:
                current_cost = new_cost
                permutation = new_permutation

    print(f"[Replication {repl+1}] permutation: {permutation} / cost: {current_cost}")


/var/folders/w7/rxr_4h1n0qv8yxb8kh1362t80000gn/T/ipykernel_74569/1987060754.py:41: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  costs = np.random.uniform(0,1,np.math.factorial(9)) * 115 + 10


[Replication 1] permutation: [0, 6, 1, 7, 3, 5, 2, 4, 8, 9] / cost: 338.8707852946092
[Replication 2] permutation: [0, 6, 7, 1, 5, 2, 3, 4, 8, 9] / cost: 334.7464277634031
[Replication 3] permutation: [0, 2, 5, 4, 3, 6, 1, 7, 8, 9] / cost: 375.5078962953237
[Replication 4] permutation: [0, 6, 5, 1, 7, 3, 2, 4, 8, 9] / cost: 289.2746793414218
[Replication 5] permutation: [0, 6, 5, 1, 7, 3, 2, 4, 8, 9] / cost: 289.2746793414218
